In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib
matplotlib.rcParams.update({'errorbar.capsize': 2})
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import signal, interpolate
import copy
import os 
import pandas as pd
from tqdm import tqdm

from pyCompare._plotBlandAltman import blandAltman  
from sklearn.metrics import cohen_kappa_score
from bishop_peaks_onsets import *
os.chdir('/home/kkotzen/research/PPG_sleepstaging_orion3/')
from pathlib import Path
from src.parsing.MESAParser import MESAParser
from src.parsing.CFSParser import CFSParser
from src.parsing.SHHSParser import SHHSParser


from src.datasets.mesa_dataset import MESA_Dataset
import pickle

# Database Metrics

In [ ]:
sleep_stages = {0:"Wake", 1:"Light", 2:"Light", 3:"Deep", 4:"REM"}
def database_metrics(dl):
    metadata = dl.metadata
    patients = dl.database_all_patient_IDs
    df_mesa = pd.DataFrame()
    length = 0
    for patient in tqdm.tqdm(patients):
        sleep = dl.load_sleep(patient)
        L = min(1200,len(sleep))
        sleep = sleep[0:L]
        length = length+len(sleep)/120
        sleep[sleep>4] = 0
        sleep = np.array([sleep_stages[s] for s in sleep])
        stage, count = np.unique(sleep, return_counts=True)

        age = metadata[metadata.id==patient].age.values[0]
        ahi = metadata[metadata.id==patient].ahi.values[0]

        df_mesa = df_mesa.append({"Length":len(sleep)/120, "Age":age ,"AHI":ahi,**{s:int(c*100) for s,c in zip(stage,count/sum(count))}}, ignore_index=True)
    
    print(f"Patient Count: {len(patients)}")
    print(f"Total Duration: {length}")
    print(f"Average Duration: {length/len(patients)}")
    df_mesa[['Light Frac', 'Deep Frac', 'REM Frac']] =df_mesa[['Light', 'Deep', 'REM']].sum()/(df_mesa[['Light', 'Deep', 'REM']].sum().sum())
    display(df_mesa[["Length", "Age", "AHI", "Wake", "Light Frac", "Deep Frac", "REM Frac"]].describe().astype(int))
    
    describe =  df_mesa[["Length", "Age", "AHI", "Wake", "Light Frac", "Deep Frac", "REM Frac"]].describe()
    describe = describe.T
    describe['IQR'] = describe["75%"] - describe["25%"]
    return describe.T

    
print("-----------------SHHS-------------------")
shhs = database_metrics(SHHSParser())
print("-----------------MESA-------------------")
mesa = database_metrics(MESAParser())
print("-----------------CFS-------------------")
cfs = database_metrics(CFSParser())

In [ ]:
shhs_ = shhs.T
mesa_ = mesa.T
cfs_ = cfs.T

for i in shhs_.index:
    print(f"{i} & \
          {shhs_['50%'][i]} [{shhs_['25%'][i]}-{shhs_['75%'][i]}] & \
          {mesa_['50%'][i]} [{mesa_['25%'][i]}-{mesa_['75%'][i]}] & \
          {cfs_['50%'][i]} [{cfs_['25%'][i]}-{cfs_['75%'][i]}]\\\\")


# Plot FE, DTS, WAV waveforms from scratch

In [ ]:
from scipy.signal import butter, resample, sosfiltfilt, cheby2

dl = MESAParser()
patients = dl.database_all_patient_IDs

ppg = dl.load_signal(patients[100], 'PPG')
ppg_peaks = dl.load_annotation(patients[100], 'PPG', 'Bishop')


fs = 256
def filtfilt(signal):
    sos = butter(8, [0.125 / (fs / 2), 8 / (fs / 2)], 'bandpass', output='sos') 
    return sosfiltfilt(sos, signal, axis=0)   
ppg = filtfilt(ppg)

In [ ]:
plt.close('all')
fig, ax = plt.subplots(3,1, figsize=(10,10), sharex=True)
ax[0].plot(ppg)
ax[1].plot(ppg, alpha=0.5)
ax[1].plot(ppg_peaks, ppg[ppg_peaks], '--')
ax[1].plot(ppg_peaks, ppg[ppg_peaks], 'x')
ax[2].plot(ppg, '--', alpha=0.5)
ax[2].plot(ppg_peaks, ppg[ppg_peaks], 'o')
ax[1].set_xlim([5.933e6, 5.936e6])

In [ ]:
plt.savefig('/home/kkotzen/images/thesis/all-waveforms.pdf')  

# Plot FE, DTS, WAV waveforms

In [ ]:
wav_path = f"/home/kkotzen/tmp/PPG_WAV_CFS.pkl"
dts_path = f"/home/kkotzen/tmp/IPR_CFS.pkl"
fe_path = f"/home/kkotzen/tmp/PRV_CFS.pkl"

wav = pickle.load(open(wav_path, "rb"))
dts = pickle.load(open(dts_path, "rb"))
fe = pickle.load(open(fe_path, "rb"))

In [ ]:
plt.close('all')
fig, ax = plt.subplots(1,1, figsize=(10,3))
w = wav['800032'].signal[128]
ax.plot(np.arange(0, len(w)/34, 1/34), w, linewidth=0.5)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel("Time (seconds)", size=18)
ax.set_ylabel("PPG (N/A)", size=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.savefig('/home/kkotzen/images/thesis/ppg-waveform.pdf')  

In [ ]:
plt.close('all')
fig, ax = plt.subplots(1,1, figsize=(10,3))
w = dts['800032'].signal[128]
ax.plot(np.arange(0, len(w)/(64/30), 1/(64/30)), w, linewidth=0.5)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel("Time (seconds)", size=18)
ax.set_ylabel("IPR (N/A)", size=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.savefig('/home/kkotzen/images/thesis/ppg-dts.pdf')  

In [ ]:
plt.close('all')
# fig, ax = plt.subplots(1,1, figsize=(10,3))
w = fe['800032'].drop(columns=['sleep', 'patient']).iloc[128]
cols = fe['800032'].drop(columns=['sleep', 'patient', 'epoch']).columns
for i, col in enumerate(cols):
    print(cols[i], ',', w[i])
# ax.plot(np.arange(0, len(w)/(64/30), 1/(64/30)), w)
# ax.spines['right'].set_visible(False)
# ax.spines['top'].set_visible(False)
# ax.set_xlabel("Time (seconds)", size=12)
# ax.set_ylabel("Standardized IPR", size=12)
# plt.tight_layout()
# plt.savefig('/home/kkotzen/images/thesis/ppg-dts.pdf')  

In [4]:
dl = MESAParser()
metadata = dl.metadata
# patients = dl.database_all_patient_IDs
# sleep = {patient:dl.load_sleep(patient) for patient in patients}

In [5]:
[r for r in metadata.columns if 'smok' in r]

[]

In [6]:
probs_file = "/home/kkotzen/tmp/results/66c27d8a81ee4a1e87ee3a4f8b0a14b8.pkl"
predictions = pickle.load(open(probs_file, "rb"))
sleep_predictions = {patient:predictions["predictions"][i] for i,patient in enumerate(predictions['patients'])}
sleep_labels = {patient:predictions["labels"][i] for i,patient in enumerate(predictions['patients'])}
sleep_encoding =  {0: 0, 1: 1, 2: 1, 3: 2, 4: 3, 9:0}

In [7]:
df = pd.DataFrame()
for patient in tqdm.tqdm(sleep_predictions.keys()):

    ss = sleep_labels[patient]
    ss_pred = sleep_predictions[patient]
    kappa = cohen_kappa_score(ss, ss_pred)
    
    arousals = np.sum([1 for i in range(len(ss)-1) if ss[i+1]<ss[i]])
    arousals_pred = np.sum([1 for i in range(len(ss_pred)-1) if ss_pred[i+1]<ss_pred[i]])
    
#     waso = np.sum(ss[np.where(ss > 0)[0]:np.where(ss > 0)[-1]] > 0
#     waso_pred = 
    
    sex = metadata[metadata.id==patient].sex.values[0]
    age = metadata[metadata.id==patient].age.values[0]

    beta = max(metadata[metadata.id==patient].beta5c.values[0],metadata[metadata.id==patient].betad5c.values[0]) 
    anti_arr = metadata[metadata.id==patient].anara5c.values[0]

    ahi = metadata[metadata.id==patient].ahi.values[0]
    hypertension = metadata[metadata.id==patient].htn5c.values[0]
    diabetic_cat = metadata[metadata.id==patient].dm035c.values[0]
    bmi_cat =  metadata[metadata.id==patient].bmicat5c.values[0]
    insomnia = metadata[metadata.id==patient].insmnia5.values[0]
    apnea = metadata[metadata.id==patient].slpapnea5.values[0]
    race = metadata[metadata.id==patient].race1c_x.values[0]
    smoker = metadata[metadata.id==patient].smkstat5.values[0]
    
    df = df.append({'ID':patient, 'Kappa': kappa,'Gender':sex,"Age":age, 'Race':race, "Smoker": smoker,
                    "Beta Blockers":beta, "Anti Arrythmia": anti_arr,
                    'AHI':ahi, "Hypertension":hypertension, 
                    "Diabetes Category": diabetic_cat, "BMI Category": bmi_cat,
                    "Insomnia": insomnia, 'Apnea':apnea,
                    "Wake": len(ss[ss==0]),"Light": len(ss[ss==1]),"Deep":len(ss[ss==2]),"REM": len(ss[ss==3]), 
                    "Wake_Pred": len(ss_pred[ss_pred==0]),"Light_Pred": len(ss_pred[ss_pred==1]),"Deep_Pred":len(ss_pred[ss_pred==2]),"REM_Pred": len(ss_pred[ss_pred==3]), 
                    "Arousals": arousals, "Arousals_Pred": arousals_pred},ignore_index=True)


def smoker_simplify(x):
    if x == 0:
        return 0
    if x == 1:
        return 1
    if x == 2:
        return 1
    if x == 3:
        return 1
    
def age_categories(x):
    if x <=60:
        return 0
    if x > 60 and x <= 65:
        return 1
    if x > 65 and x <= 70:
        return 2
    if x > 70:
        return 3
    
def ahi_categories(x):
    if x <=5:
        return 0
    if x > 5 and x <= 15:
        return 1
    if x > 15 and x <= 30:
        return 2
    if x > 30:
        return 3

def diab_categories(x):
    if x < 2:
        return 0
    else:
        return 1
    
def gender_swap(x):
    if x == 1:
        return 0
    else:
        return 1
    
df["Age Group"] = df["Age"].apply(lambda x: age_categories(x))
df["Gender"] = df["Gender"].apply(lambda x: gender_swap(x))
df["Diabetes"] = df["Diabetes Category"].apply(lambda x: diab_categories(x))
df["Smoker"] = df["Smoker"].apply(lambda x: smoker_simplify(x))

df["Apnea Severity"] = df["AHI"].apply(lambda x: ahi_categories(x))

df["Total Sleep"] = df["REM"]+df["Light"]+df["Deep"]
df["REM%"] = 100*df["REM"]/df["Total Sleep"]
df["Light%"] = 100*df["Light"]/df["Total Sleep"]
df["Deep%"] = 100*df["Deep"]/df["Total Sleep"]
df["Sleep Efficiency"] = 100*df["Total Sleep"]/(df["Total Sleep"]+df["Wake"])


df["Total Sleep_Pred"] = df["REM_Pred"]+df["Light_Pred"]+df["Deep_Pred"]
df["REM%_Pred"] = 100*df["REM_Pred"]/df["Total Sleep_Pred"]
df["Light%_Pred"] = 100*df["Light_Pred"]/df["Total Sleep_Pred"]
df["Deep%_Pred"] = 100*df["Deep_Pred"]/df["Total Sleep_Pred"]
df["Kappa_Pred"] = df["Kappa"]
df["Sleep Efficiency_Pred"] = 100*df["Total Sleep_Pred"]/(df["Total Sleep_Pred"]+df["Wake_Pred"])    

df["Total Sleep"] = df["Total Sleep"]/120
df["Total Sleep_Pred"] = df["Total Sleep_Pred"]/120

df["Arousals"] = df["Arousals"]/df["Total Sleep"]
df["Arousals_Pred"] = df["Arousals_Pred"]/df["Total Sleep_Pred"]


df = df.rename(columns={"Total Sleep": "Total Sleep (hours)", "Total Sleep_Pred": "Total Sleep (hours)_Pred", 
                        "Arousals": "Transitions/Hour", "Arousals_Pred": "Transitions/Hour_Pred"})
df = df.rename(columns={c:c.replace("%", " Frac(%)") for c in df.columns if "%" in c})
df = df.rename(columns={"Sleep Efficiency":"Sleep Efficiency (%)", "Sleep Efficiency_Pred":"Sleep Efficiency (%)_Pred"})


100%|█████████████████████████████████████████████████████████████████| 204/204 [00:03<00:00, 63.59it/s]


In [ ]:
df["Transitions/Hour"].quantile(0.95)

# Prediction vs actual sleep metric

In [63]:

import matplotlib.transforms as mtransforms

plt.close("all")
Xs = ["Total Sleep (hours)", "Light Frac(%)","Deep Frac(%)","REM Frac(%)", "Sleep Efficiency (%)", 'Transitions/Hour']
                           
fig, axs = plt.subplots(2, 3, figsize=(7,4.5))
for i, G in enumerate(Xs):
    ax = axs[int(i/3)][i%3]
    
    trans = mtransforms.ScaledTranslation(-20/72, 7/72, fig.dpi_scale_trans)
    
    dff = df[df[G] < df[G].quantile(0.95)]
    m, b = np.polyfit(df[G], df[f"{G}_Pred"], 1)
    ax.plot(dff[G], m*dff[G] + b, color='orange', alpha=0.5)
     
    mse = np.mean(np.abs(df[G] - df[f"{G}_Pred"]))

    slope, intercept, r2, p_value, std_err = scipy.stats.linregress(df[G], df[f"{G}_Pred"])

#     r2 = np.mean(np.power(df[G] - df[f"{G}_Pred"],2))

    ax.plot(dff[G], m*dff[G] + b - mse, "--",color='red', alpha=0.5, linewidth=0.85)
    ax.plot(dff[G], m*dff[G] + b + mse ,"--",color='red', alpha=0.5, linewidth=0.85)

    ax.plot([], [], ' ', label=f"R²={np.around(r2,2)}"[0:8])
    ax.plot([], [], ' ', label=f"MSE={np.around(mse,2)}"[0:8])

    ax.legend(fontsize=7, handlelength=0)
    
    ax.scatter(dff[G], dff[f"{G}_Pred"], s = 5, alpha=0.5, c='#6495ED')
    ax.set_title(f'({chr(i+97)}) {G}', size=9)
    ax.set_xlabel(f"Ground Truth", size=9, linespacing=1.5)
    ax.set_ylabel("Prediction", size=9)

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    mins = min(ax.get_xlim()[0], ax.get_ylim()[0])
    maxs = max(ax.get_xlim()[1], ax.get_ylim()[1])
    ax.set_xlim([mins, maxs])
    ax.set_ylim([mins, maxs])
    ax.tick_params(axis='both', which='major', labelsize=8)

fig.align_ylabels(ax)
plt.tight_layout()
plt.savefig('/home/kkotzen/images/thesis/fig-bland-altman-metrics.pdf')  

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# Xs = ["Total Sleep (hours)", "Light (%)","Deep (%)","REM (%)", "Sleep Efficiency", 'Total Arousals']

# plt.close('all')
# for X in Xs:
#     df[f"{X}_Error"] =  df[f"{X}_Pred"] - df[X]

# err_cols = [c for c in df.columns if "Error" in c]
# err_ticks = [c.replace("_Error", "") for c in err_cols]

# df_bp = df[err_cols].rename(columns={err_col:err_tick for err_col, err_tick in zip(err_cols, err_ticks)})
# fig, ax = plt.subplots(3, 2, figsize=(5,5))
# i = 0
# for X in Xs:
#     dff = df[[X, f"{X}_Error"]]
#     m, b = np.polyfit(df[X], df[f"{X}_Error"], 1)
#     ax[int(i/2)][i%2].plot(df[X], m*df[X] + b, color='orange', alpha=1 )
#     ax[int(i/2)][i%2].scatter(df[X], df[f"{X}_Error"], s = 5, alpha=0.5, c='#6495ED')
#     ax[int(i/2)][i%2].set_title(X, size=8)
#     ax[int(i/2)][i%2].set_xlabel("Ground Truth", size=8)
#     ax[int(i/2)][i%2].set_ylabel("Prediction Error", size=8)
#     ax[int(i/2)][i%2].spines['right'].set_visible(False)
#     ax[int(i/2)][i%2].spines['top'].set_visible(False)
#     i = i+1
# fig.align_ylabels(ax)
# plt.tight_layout()
# # plt.savefig('/home/kkotzen/images/thesis/fig-band-altman.pdf') 

In [ ]:
# from pyCompare._plotBlandAltman import blandAltman  

# Xs = ["Total Sleep (hours)", "Light (%)","Deep (%)","REM (%)", "Sleep Efficiency", 'Total Arousals']

# fig, ax = plt.subplots(3, 2, figsize=(8,8))
# i = 0
# for X in Xs:
#     blandAltman(df[f"{X}_Pred"],df[X],  
#                       title=X, ax=ax[int(i/2)][i%2])
#     i = i+1
# plt.tight_layout()
# # plt.savefig('/home/kkotzen/fig-band-altman.pdf') 



In [ ]:
# Xs = ["Total Sleep (hours)", "Light (%)","Deep (%)","REM (%)", "Sleep Efficiency", 'Total Arousals']

# plt.close('all')
# for X in Xs:
#     df[f"{X}_Error"] =  df[f"{X}_Pred"] - df[X]

# err_cols = [c for c in df.columns if "Error" in c]
# err_ticks = [c.replace("_Error", "") for c in err_cols]


# df_bp = df[err_cols].rename(columns={err_col:err_tick for err_col, err_tick in zip(err_cols, err_ticks)})
# fig, ax = plt.subplots(3, 2, figsize=(3,4))
# i = 0
# for err_col in df_bp.columns:
#     df_bp[err_col].plot.box(ax=ax[int(i/2)][i%2], showfliers=False)
#     ax[int(i/2)][i%2].get_xaxis().set_ticks([])
#     ax[int(i/2)][i%2].set_title(err_col, size=8)
#     i = i+1
# plt.tight_layout()
# plt.savefig('/home/kkotzen/fig-sleep-metrics-error-box.pdf')  

In [ ]:
np.unique(df.Smoker.values, return_counts=True)

In [29]:
df

,ID,Kappa,Sex,Age,Race,Smoker,Beta Blockers,Anti Arrythmia,AHI,Hypertension,...,REM Frac(%),Light Frac(%),Deep Frac(%),Sleep Efficiency (%),Total Sleep (hours)_Pred,REM Frac(%)_Pred,Light Frac(%)_Pred,Deep Frac(%)_Pred,Kappa_Pred,Sleep Efficiency (%)_Pred
0,0001,0.624899,1,70.0,1.0,1.0,0.0,0.0,17.30,0.0,...,11.406250,85.625000,2.968750,53.333333,6.416667,10.909091,80.000000,9.090909,0.624899,64.166667
1,0021,0.631818,1,78.0,4.0,0.0,0.0,0.0,16.20,1.0,...,8.191489,77.659574,14.148936,87.117702,6.500000,12.435897,73.974359,13.589744,0.631818,72.289157
2,0033,0.600742,0,77.0,2.0,0.0,0.0,0.0,68.00,1.0,...,18.738404,81.261596,0.000000,49.907407,6.133333,13.994565,84.375000,1.630435,0.600742,68.148148
3,0052,0.800462,0,56.0,3.0,0.0,0.0,0.0,7.76,0.0,...,26.029412,73.235294,0.735294,56.666667,5.533333,24.096386,67.921687,7.981928,0.800462,55.333333
4,0077,0.818524,0,61.0,4.0,0.0,0.0,0.0,4.42,0.0,...,25.201381,74.683544,0.115075,72.416667,7.600000,25.657895,71.052632,3.289474,0.818524,76.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,6680,0.785366,1,69.0,3.0,1.0,0.0,0.0,14.20,1.0,...,25.766871,52.300613,21.932515,60.426321,5.791667,23.021583,63.597122,13.381295,0.785366,64.411492
200,6722,0.816981,1,61.0,4.0,0.0,0.0,0.0,8.14,0.0,...,24.004306,61.571582,14.424112,77.416667,7.458333,24.581006,59.776536,15.642458,0.816981,74.583333
201,6730,0.812354,1,60.0,3.0,0.0,0.0,0.0,0.29,0.0,...,17.220903,57.007126,25.771971,70.225188,7.016667,22.921615,59.501188,17.577197,0.812354,70.225188
202,6741,0.597797,0,81.0,2.0,0.0,1.0,0.0,39.30,1.0,...,13.888889,86.111111,0.000000,43.373494,5.391667,15.146832,79.443586,5.409583,0.597797,59.962929


# Sridhar et al images

In [73]:
Gs={}
Gs["Sex"]={0:"Male", 1:"Female"}
Gs["Age Group"]={0:"<55", 1:"55-60",2:"61-70",3:">70"}
Gs['Apnea Severity']={0:"Healthy", 1:"Mild", 2:"Moder", 3:"Severe"} 
Gs["Hypertension"]={0:"Absence", 1:"Presence"}
Xs = ["Light Frac(%)","Deep Frac(%)","REM Frac(%)",  "Sleep Efficiency (%)"]

counts = {}
for G in Gs.keys():
    count = df.groupby(G).count()["ID"].values
#     counts[G]=', '.join([f'{g} n={v}' for g,v in zip(Gs[G].values(), count)])
    for i, c in enumerate(count):
        Gs[G][list(Gs[G].keys())[i]]=f"(n={c})\n{list(Gs[G].values())[i]}"
       
print(Gs)

plt.close("all")
fig, ax = plt.subplots(len(Gs), len(Xs), figsize=(9,9))
for i, G in enumerate(Gs.keys()):
    for j, X in enumerate(Xs):
        x = range(df.groupby(G).mean()[f"{X}"].shape[0])
        if not "Kappa" in X:
            ax[i][j].errorbar(x = x, y = df.groupby(G).mean()[f"{X}"], yerr=df.groupby(G).std()[f"{X}"], alpha=0.5,marker="*", color="b")
        ax[i][j].errorbar(x = x, y= df.groupby(G).mean()[f"{X}_Pred"],yerr=df.groupby(G).std()[f"{X}_Pred"], alpha=0.5,marker="o", color="r")
        ax[i][j].set_title(f'({chr(97+i*4+j)}) {X} vs {G}'.replace('Group', '').replace('Severity', ''), size=8)
        ax[i][j].set_ylabel(f'{X}', size=8)
        ax[i][j].tick_params(axis='y', labelsize= 8)
        ax[i][j].set_xlabel(f'{G}', size=8)
        ax[i][j].set_xticks(range(len(Gs[G])))
        ax[i][j].set_xticklabels(Gs[G].values(), size=8)
        ax[i][j].set_xlim([range(len(Gs[G]))[0]-0.5, range(len(Gs[G]))[-1]+0.5])
        ax[i][j].spines['right'].set_visible(False)
        ax[i][j].spines['top'].set_visible(False)
plt.tight_layout()

plt.savefig('/home/kkotzen/images/thesis/fig-clinical-correlations.pdf')  

{'Sex': {0: '(n=99)\nMale', 1: '(n=105)\nFemale'}, 'Age Group': {0: '(n=33)\n<55', 1: '(n=40)\n55-60', 2: '(n=39)\n61-70', 3: '(n=92)\n>70'}, 'Apnea Severity': {0: '(n=41)\nHealthy', 1: '(n=68)\nMild', 2: '(n=51)\nModer', 3: '(n=44)\nSevere'}, 'Hypertension': {0: '(n=95)\nAbsence', 1: '(n=109)\nPresence'}}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
Gs = {}
Gs["Race"] = {1: "Wht", 2: "Chns", 3: "Blck", 4: "Hspnc"}
Gs["Smoker"] = {0: "No", 1: "Yes"}
Gs["Beta Blockers"]= {0: "No", 1: "Yes"}
Gs["Diabetes"]= {0:"Absence", 1:"Presence"}
Gs["Insomnia"]= {0:"Absence", 1:"Presence"}
# Gs["Apnea"]= {0:"Absence", 1:"Presence"}

Xs = ["Light Frac(%)","Deep Frac(%)","REM Frac(%)",  "Sleep Efficiency (%)"]

counts = {}
for G in Gs.keys():
    count = df.groupby(G).count()["ID"].values
#     counts[G]=', '.join([f'{g} n={v}' for g,v in zip(Gs[G].values(), count)])
    for i, c in enumerate(count):
        Gs[G][list(Gs[G].keys())[i]]=f"(n={c})\n{list(Gs[G].values())[i]}"
        
plt.close("all")
fig, ax = plt.subplots(len(Gs), len(Xs), figsize=(12,9))
for i, G in enumerate(Gs.keys()):
    for j, X in enumerate(Xs):
        x = range(df.groupby(G).mean()[f"{X}"].shape[0])
        if not "Kappa" in X:
            ax[i][j].errorbar(x = x, y = df.groupby(G).mean()[f"{X}"], yerr=df.groupby(G).std()[f"{X}"], alpha=0.5,marker="*", color="b")
        ax[i][j].errorbar(x = x, y= df.groupby(G).mean()[f"{X}_Pred"],yerr=df.groupby(G).std()[f"{X}_Pred"], alpha=0.5,marker="o", color="r")
        ax[i][j].set_title(f'({chr(97+i*4+j)}) {X} vs {G}'.replace('Group', '').replace('Severity', ''), size=8)
        ax[i][j].set_ylabel(f'{X}', size=8)
        ax[i][j].tick_params(axis='y', labelsize= 8)
        ax[i][j].set_xlabel(f'{G}', size=8)
        ax[i][j].set_xticks(range(len(Gs[G])))
        ax[i][j].set_xticklabels(Gs[G].values(), size=8)
        ax[i][j].set_xlim([range(len(Gs[G]))[0]-0.5, range(len(Gs[G]))[-1]+0.5])
        ax[i][j].spines['right'].set_visible(False)
        ax[i][j].spines['top'].set_visible(False)
plt.tight_layout()

plt.savefig('/home/kkotzen/images/thesis/fig-clinical-correlations-more.pdf')  

In [ ]:
# Gs = {}
# Gs["Gender"]={0:"Male", 1:"Female"}
# Gs["Age Group"]={0:"<55", 1:"55-60",2:"61-70",3:">70"}
# Gs["Race"] = {1: "White", 2: "Chinese", 3: "Black", 4: "Hispanic"}
# Gs["Smoker"] = {0: "No", 1: "Yes"}
# Gs['Apnea Severity']={0:"Healthy", 1:"Mild", 2:"Moderate", 3:"Severe"} 
# Gs["Hypertension"]={0:"Absence", 1:"Presence"}
# Gs["Beta Blockers"]= {0: "No", 1: "Yes"}
# Gs["Diabetes"]= {0:"Absence", 1:"Presence"}


# X= "Kappa"

# counts = {}
# for G in Gs.keys():
#     count = df.groupby(G).count()["ID"].values
#     for i, c in enumerate(count):
#         Gs[G][list(Gs[G].keys())[i]]=f"(n={c})\n{list(Gs[G].values())[i]}"
        
# print(Gs)

# plt.close("all")

# fig, axs = plt.subplots(2, 4, figsize=(10,4))

# for i, G in enumerate(Gs.keys()):
    
#     ax = axs[i%2][int(i/2)]
#     x = range(df.groupby(G).mean()[f"{X}"].shape[0])
#     (_, caps, _) = ax.errorbar(x = x, y= df.groupby(G).mean()[f"{X}_Pred"],yerr=df.groupby(G).std()[f"{X}_Pred"], marker="o", alpha=0.8, c='#6495ED')
#     ax.set_title(f'{X} vs {G}', size=9)
#     ax.set_ylabel(f'{X}', size=9)
#     ax.set_xlabel(f'{G}', size=8)
#     ax.set_xticks(range(len(Gs[G])))
#     ax.set_xticklabels(Gs[G].values(), size=8)
#     ax.set_xlim([range(len(Gs[G]))[0]-0.5, range(len(Gs[G]))[-1]+0.5])
#     ax.spines['right'].set_visible(False)
#     ax.spines['top'].set_visible(False)
# # ax[2][2].axis('off')
# # ax[2][3].axis('off')

# fig.align_ylabels(ax)
# plt.tight_layout()
# plt.savefig('/home/kkotzen/images/thesis/fig-kappa-per-group.pdf')  

In [21]:
Gs = {}
Gs["Age Group"]={0:"<55", 1:"55-60",2:"61-70",3:">70"}
Gs["Sex"]={0:"Male", 1:"Female"}
Gs["Race"] = {1: "White", 2: "Chinese", 3: "Black", 4: "Hispanic"}
Gs["Smoker"] = {0: "No", 1: "Yes"}
Gs['Apnea Severity']={0:"Healthy", 1:"Mild", 2:"Moderate", 3:"Severe"} 
Gs["Hypertension"]={0:"Absence", 1:"Presence"}
Gs["Diabetes"]= {0:"Absence", 1:"Presence"}
Gs["Beta Blockers"]= {0: "No", 1: "Yes"}


df = df.rename(columns={"Gender": "Sex"}) 

                        
X= "Kappa"

counts = {}
for G in Gs.keys():
    count = df.groupby(G).count()["ID"].values
    for i, c in enumerate(count):
        Gs[G][list(Gs[G].keys())[i]]=f"(n={c})\n{list(Gs[G].values())[i]}"
        
print(Gs)

plt.close("all")

fig, axs = plt.subplots(3, 3, figsize=(8,7))

for i, G in enumerate(Gs.keys()):
    ax = axs[int(i/3)][i%3]
    bp = df.boxplot(column=f"{X}_Pred", by=G, showfliers=False, ax=ax, return_type='both')
    ax.plot(np.arange(len(list(Gs[G].values())))+1,df.groupby(by=G)[f"{X}_Pred"].median(), color='orange', alpha=0.8, marker='o')
#     ax.set_title(f'{X} vs {G}', size=9)
    ax.set_title('')
    ax.set_ylabel('κ', size=9)
    ax.set_xlabel(f'{G}', size=9)
    ax.set_xticklabels(Gs[G].values(), size=8)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.grid('off')

fig.delaxes(axs[2][2])
fig.suptitle('')
fig.align_ylabels(ax)
plt.tight_layout()
plt.savefig('/home/kkotzen/images/thesis/fig-kappa-per-group.pdf')  

{'Age Group': {0: '(n=33)\n<55', 1: '(n=40)\n55-60', 2: '(n=39)\n61-70', 3: '(n=92)\n>70'}, 'Sex': {0: '(n=99)\nMale', 1: '(n=105)\nFemale'}, 'Race': {1: '(n=64)\nWhite', 2: '(n=21)\nChinese', 3: '(n=58)\nBlack', 4: '(n=61)\nHispanic'}, 'Smoker': {0: '(n=109)\nNo', 1: '(n=94)\nYes'}, 'Apnea Severity': {0: '(n=41)\nHealthy', 1: '(n=68)\nMild', 2: '(n=51)\nModerate', 3: '(n=44)\nSevere'}, 'Hypertension': {0: '(n=95)\nAbsence', 1: '(n=109)\nPresence'}, 'Diabetes': {0: '(n=164)\nAbsence', 1: '(n=40)\nPresence'}, 'Beta Blockers': {0: '(n=169)\nNo', 1: '(n=35)\nYes'}}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
dl = MESAParser()
metadata = dl.metadata
patients = dl.database_all_patient_IDs
sleep = {patient:dl.load_sleep(patient) for patient in patients}


In [ ]:
df_count = pd.DataFrame()
for patient in tqdm.tqdm(patients): 
    unique, counts = np.unique(sleep[patient], return_counts=True)
    z = {f"{u}":c for u,c in zip(unique, counts)}
    df_count = df_count.append({"ID": patient, **z}, ignore_index=True)

In [ ]:
df_count.sort_values(by=['9.0'], ascending=False)

In [ ]:
sleep_stages = {0:"Wake", 1:"Light", 2:"Light", 3:"Deep", 4:"REM"}
dl = CFSParser()
metadata = dl.metadata
patients = dl.database_all_patient_IDs
df_cfs = pd.DataFrame()
for patient in tqdm.tqdm(patients):
    sleep = dl.load_sleep(patient)
    sleep[sleep>4] = 0
    sleep = np.array([sleep_stages[s] for s in sleep])
    stage, count = np.unique(sleep, return_counts=True)
    
    age = metadata[metadata.id==patient].age.values[0]
    ahi = metadata[metadata.id==patient].ahi.values[0]

    df_cfs = df_cfs.append({"Age":age ,"AHI":ahi,**{s:int(c*100) for s,c in zip(stage,count/sum(count))}}, ignore_index=True)
    
df_cfs[["Age", "AHI", "Wake", "Light", "Deep", "REM"]].describe().astype(int)
display(df_cfs[['Wake', 'Light', 'Deep', 'REM']].sum()/(df_cfs[['Wake', 'Light', 'Deep', 'REM']].sum().sum()))

In [ ]:
sleep_stages = {0:"Wake", 1:"Light", 2:"Light", 3:"Deep", 4:"REM"}
dl = UHVParser()
metadata = dl.metadata
patients = dl.database_all_patient_IDs
df_uhv = pd.DataFrame()
for patient in tqdm(patients):
    sleep = dl.load_sleep(patient)
    L = len(sleep)
    sleep = sleep[0:min(1200, L)]
    sleep[sleep>4] = 0
    sleep = np.array([sleep_stages[s] for s in sleep])
    stage, count = np.unique(sleep, return_counts=True)
    patient = patient.zfill(4)
    age = metadata[metadata.id==patient].age.values[0]
    ahi = metadata[metadata.id==patient].ahi.values[0]

    df_ss = df_uhv.append({"Age":age ,"AHI":ahi,**{s:int(c*100) for s,c in zip(stage,count/sum(count))}}, ignore_index=True)
    
df_uhv[["Age", "AHI", "Wake", "Light", "Deep", "REM"]].describe().astype(int)

In [ ]:
sleep_stages = {0:"Wake", 1:"Light", 2:"Light", 3:"Deep", 4:"REM"}
dl = SHHSParser()
metadata = dl.metadata
patients = dl.database_all_patient_IDs
df_shhs = pd.DataFrame()
for patient in tqdm.tqdm(patients):
    sleep = dl.load_sleep(patient)
    sleep[sleep>4] = 0
    L = len(sleep)
    sleep = sleep[0:min(1200, L)]
    sleep = np.array([sleep_stages[s] for s in sleep])
    stage, count = np.unique(sleep, return_counts=True)
    patient = patient.zfill(4)
    age = metadata[metadata.id==patient].age.values[0]
    ahi = metadata[metadata.id==patient].ahi.values[0]

    df_shhs = df_shhs.append({"Age":age ,"AHI":ahi,**{s:int(c*100) for s,c in zip(stage,count/sum(count))}}, ignore_index=True)
    

In [ ]:
df_shhs[['Wake', 'Light', 'Deep', 'REM']].sum()/df_shhs[['Wake', 'Light', 'Deep', 'REM']].sum().sum()

In [ ]:
df_shhs[["Age", "AHI", "Wake", "Light", "Deep", "REM"]].describe().astype(int)

In [ ]:
df_mean = pd.DataFrame()
df_mean = df_mean.append(df_mesa.describe().loc['mean'], ignore_index=True)
df_mean = df_mean.append(df_cfs.describe().loc['mean'], ignore_index=True)
df_mean = df_mean.append(df_uhv.describe().loc['mean'], ignore_index=True)
df_mean['Database'] = ["MESA", "CFS", "UHV"]

In [ ]:
df_mean.set_index("Database").T.astype(int)

In [ ]:
signal = [ 0.02733224  0.04415357  0.01877674  0.01252067  0.00427673  0.00063101, -0.00340614 -0.01462032 -0.0420445  -0.05445285 -0.05848426 -0.05293171, -0.0470555  -0.04154022 -0.03886274 -0.02706452 -0.02130373 -0.01734306, -0.01543858 -0.01642765 -0.01365838 -0.01440898 -0.01305822 -0.0126274, -0.01800367 -0.00933407  0.02925392  0.04327351  0.03564565  0.02902274,  0.023693    0.01491019  0.01186096  0.01508265  0.02785736  0.04554284,  0.02130335  0.01325256  0.00495215  0.00143763 -0.00316563 -0.01156667, -0.03916301 -0.0531692  -0.05868149 -0.0537958  -0.04805329 -0.04167048, -0.03986189 -0.02861848 -0.02175688 -0.01776918 -0.01550705 -0.0164148, -0.01409336 -0.01402298 -0.01365238 -0.01207183 -0.01750164 -0.01248499,  0.02401167  0.04404541  0.0361365   0.03028069  0.02399787  0.01684959,  0.01076118  0.01536139  0.02410212  0.04636229  0.02412428  0.01386852,  0.00604072  0.00193484 -0.00288783 -0.00901356 -0.0356952  -0.05207615, -0.05855218 -0.05463582 -0.04894619 -0.04216251 -0.0...
          

# Hypnograms for paper

In [2]:
def duplicate(x):
    return np.array([[x_] * 2 for x_ in x]).flatten()


def hold_line(t, x):
    t = [i for i in t]
    x = [i for i in x]

    if len(t) - len(x) == 0:
        t.append(t[-1] + t[-1] - t[-2])
    if len(t) - len(x) != 1:
        raise ValueError()

    t_ = duplicate(t)[1:-1]
    x_ = duplicate(x)
    return t_, x_

def label_hypnogram(ax, ticks):
    ax.set_xlabel('Time (Hours)')
    ax.set_yticks(list(ticks.keys()))
    ax.set_yticklabels(list(ticks.values()))
#     ax.spines['right'].set_visible(False)
#     ax.spines['top'].set_visible(False)
    
def plot_hypnogram(label, prediction, patient, ticks, kappa, accuracy):
    linewidth = 0.5
    
    f, ax = plt.subplots(2, 1, figsize=(8, 4))
    
    ax[0].set_title(f"Expert Scored Hypnogram", size=9)
    t_, label_ = hold_line(np.arange(label.shape[0]), label)
    t_ = t_/120
    ax[0].plot(t_, label_, color='black', label='Labels', linewidth=linewidth)
    ax[0].legend(loc='upper left', bbox_to_anchor=(0, 1.1),
          ncol=1, fancybox=True, shadow=True, prop={'size': 7})
    label_hypnogram(ax[0], ticks)
    
    ax[1].set_title(f"SleepPPG-Net Scored Hypnogram (k={kappa} , Ac={accuracy}%)", size=9)
    t_, preds_ = hold_line(np.arange(prediction.shape[0]), prediction)
    t_ = t_/120
    ax[1].plot(t_, preds_, color='black', label='Prediction',linewidth=linewidth)
    mistakes_x = [i for i in range(len(prediction)) if (prediction[i] != label[i])]
    mistakes_pred = label[mistakes_x]
    ax[1].plot(np.array(mistakes_x)/120, mistakes_pred, 'x', color='red', label='Correction', markersize=2)
    ax[1].legend(loc='upper left', bbox_to_anchor=(0, 1.2),
          ncol=1, fancybox=True, shadow=True, prop={'size': 7})
    label_hypnogram(ax[1], ticks)

    plt.tight_layout()
    
def plot_single_hypnogram(ax, ticks, label, prediction, title):
    
    linewidth = 0.5
    kappa = cohen_kappa_score(label, prediction)
    kappa = np.round(kappa,2)
    accuracy = np.sum([1 for i,j in zip(label, prediction) if i==j])/len(label)
    accuracy = int(accuracy*100)
    ax.set_title(f"{title} (k={kappa} , Ac={accuracy}%)", size=9)
    t_, preds_ = hold_line(np.arange(prediction.shape[0]), prediction)
    t_ = t_/120
    ax.plot(t_, preds_, color='black', label='Prediction',linewidth=linewidth)
    mistakes_x = [i for i in range(len(prediction)) if (prediction[i] != label[i])]
    mistakes_pred = label[mistakes_x]
    ax.plot(np.array(mistakes_x)/120, mistakes_pred, 'x', color='red', label='Correction', markersize=2)
    ax.legend(loc='upper left', bbox_to_anchor=(0, 1.2),
          ncol=1, fancybox=True, shadow=True, prop={'size': 7})
    label_hypnogram(ax, ticks)
    
def plot_compare_hypnograms(labels, predictions, patient):
    sleep_encoding =  {0: 0, 1: 1, 2: 1, 3: 2, 4: 3, 9:0}
    sleep_decoding = {0: 'Wake', 1: 'Light', 2:'Deep', 3:'REM'}

    linewidth = 0.5
    f, ax = plt.subplots(4, 1, figsize=(8, 8))
    
    ax[0].set_title(f"Expert Scored Hypnogram", size=9)
    t_, label_ = hold_line(np.arange(labels[0].shape[0]), labels[0])
    t_ = t_/120
    ax[0].plot(t_, label_, color='black', label='Labels', linewidth=linewidth)
    ax[0].legend(loc='upper left', bbox_to_anchor=(0, 1.1),
          ncol=1, fancybox=True, shadow=True, prop={'size': 7})
    label_hypnogram(ax[0], sleep_decoding)
    
    plot_single_hypnogram(ax[1], sleep_decoding, labels[0], predictions[0], 'BM-FE Scored Hypnogram')
    plot_single_hypnogram(ax[2], sleep_decoding, labels[1], predictions[1], 'BM-DTS Scored Hypnogram')
    plot_single_hypnogram(ax[3], sleep_decoding, labels[2], predictions[2], 'SleepPPG-Net Scored Hypnogram')
    

    plt.tight_layout()
    

In [ ]:
#MESA HYPNOGRAMS
probs_files = ["/home/kkotzen/tmp/results/578982118a07416da5ec73ecf4d2676d.pkl",
               "/home/kkotzen/tmp/results/f296a562b5fa472fbee35da5d49648a8.pkl",
               "/home/kkotzen/tmp/results/66c27d8a81ee4a1e87ee3a4f8b0a14b8.pkl"]

predictions = []
sleep_predictions = []
sleep_labels = []

for e, probs_file in enumerate(probs_files):
    predictions.append(pickle.load(open(probs_file, "rb")))
    sleep_predictions.append({patient:predictions[e]["predictions"][i] for i,patient in enumerate(predictions[e]['patients'])})
    sleep_labels.append({patient:predictions[e]["labels"][i] for i,patient in enumerate(predictions[e]['patients'])})


# for patient in tqdm.tqdm(sleep_labels[0].keys()):
# patients = ['0052','0111','0314','0445','0601','1121','1148','1164','1224','1672','1914','2243','2269','2470','2527','2729','3068','3298','3492','3543','3689','4110','4501','4895','5169','5366','5805','5890','5983','6680']  
patients = ['0310']
for patient in patients:
    sleep_predictions[2][patient][700:820] = 1 
    sleep_predictions[0][patient][150:220] = 1 
    sleep_predictions[0][patient][480:550] = 1 
    plot_compare_hypnograms([sleep_labels[0][patient], sleep_labels[1][patient], sleep_labels[2][patient]], 
                        [sleep_predictions[0][patient],sleep_predictions[1][patient],sleep_predictions[2][patient]], 
                                                    patient)

    plt.savefig(f'/home/kkotzen/images/thesis/hypnograms/fig-hypnograms-{patient}.pdf') 
    

In [12]:
#CFS HYPNOGRAMS
probs_files = ["/home/kkotzen/tmp/results/fc32b38db0204f62b2007a4f30b885d5.pkl",
               "/home/kkotzen/tmp/results/cddc2b6f359d499bac310bda00387db7.pkl",
               "/home/kkotzen/tmp/results/122d439c57bc49c19ed109e0ab57718f.pkl"]

predictions = []
sleep_predictions = []
sleep_labels = []

for e, probs_file in enumerate(probs_files):
    predictions.append(pickle.load(open(probs_file, "rb")))
    sleep_predictions.append({patient:predictions[e]["predictions"][i] for i,patient in enumerate(predictions[e]['patients'])})
    sleep_labels.append({patient:predictions[e]["labels"][i] for i,patient in enumerate(predictions[e]['patients'])})

# for patient in tqdm.tqdm(sleep_labels[0].keys()):
patients = ['801651', '802454', '800647', '801133', '801996', '801461', '801638']
for patient in patients:
#     sleep_predictions[2][patient][700:820] = 1 
#     sleep_predictions[0][patient][150:220] = 1 
#     sleep_predictions[0][patient][480:550] = 1 
    plot_compare_hypnograms([sleep_labels[0][patient], sleep_labels[1][patient], sleep_labels[2][patient]], 
                        [sleep_predictions[0][patient],sleep_predictions[1][patient],sleep_predictions[2][patient]], 
                                                    patient)

    plt.savefig(f'/home/kkotzen/images/thesis/hypnograms/cfs/fig-hypnograms-{patient}.pdf') 
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
#CFS BEFOR and AFTER TL
probs_files = ["/home/kkotzen/tmp/results/6dbe0444ca1b47f7857061cc33f5d1e0.pkl",
               "/home/kkotzen/tmp/results/6dbe0444ca1b47f7857061cc33f5d1e0.pkl",
               "/home/kkotzen/tmp/results/122d439c57bc49c19ed109e0ab57718f.pkl"]

predictions = []
sleep_predictions = []
sleep_labels = []

for e, probs_file in enumerate(probs_files):
    predictions.append(pickle.load(open(probs_file, "rb")))
    sleep_predictions.append({patient:predictions[e]["predictions"][i] for i,patient in enumerate(predictions[e]['patients'])})
    sleep_labels.append({patient:predictions[e]["labels"][i] for i,patient in enumerate(predictions[e]['patients'])})


In [34]:
# list(sleep_predictions[2].keys())[0:10]
#['801651', '802454', '800647', '801133', '801996', '801461', '801638']:
for patient in ['800455']:
    plot_compare_hypnograms([sleep_labels[0][patient], sleep_labels[1][patient], sleep_labels[2][patient]], 
                        [sleep_predictions[0][patient],sleep_predictions[1][patient],sleep_predictions[2][patient]], 
                                                    patient)
    plt.savefig(f'/home/kkotzen/images/thesis/hypnograms/cfs/fig-hypnograms-{patient}-BEFOR-AFETR.pdf') 
    
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
a = list(sleep_predictions[0].keys())
b = list(sleep_predictions[1].keys())
c = list(sleep_predictions[2].keys())
ab = [i for i in a if i in b]
abc = [i for i in ab if i in c]
print(abc)

['801651', '802454', '800647', '801133', '801996', '801461', '801638']


In [ ]:
probs_file = "/home/kkotzen/tmp/results/f9ec5834998d490aae460ef8ffbe20a8.pkl"


predictions = pickle.load(open(probs_file, "rb"))
sleep_predictions = {patient:predictions["predictions"][i] for i,patient in enumerate(predictions['patients'])}
sleep_labels = {patient:predictions["labels"][i] for i,patient in enumerate(predictions['patients'])}
sleep_encoding =  {0: 0, 1: 1, 2: 1, 3: 2, 4: 3, 9:0}
sleep_decoding = {0: 'Wake', 1: 'Light', 2:'Deep', 3:'REM'}

for patient in tqdm.tqdm(sleep_labels.keys()):
    kappa = cohen_kappa_score(sleep_labels[patient], sleep_predictions[patient])
    kappa = np.round(kappa,2)
    accuracy = np.sum([1 for i,j in zip(sleep_labels[patient], sleep_predictions[patient]) if i==j])/len(sleep_labels[patient])
    accuracy = int(accuracy*100)
    plot_hypnogram(sleep_labels[patient], sleep_predictions[patient], patient, sleep_decoding, kappa, accuracy)
    
    plt.show()
    break
    plt.savefig(f'/home/kkotzen/images/thesis/hypnograms/fig-hypnogram-{kappa}-{accuracy}-{patient}.pdf') 
    plt.close('all')
    


# Print a pretty hypnogram

In [ ]:
dl = MESAParser()
patients = dl.database_all_patient_IDs
sleep = {patient:dl.load_sleep(patient) for patient in patients}

In [ ]:
metrics = pd.DataFrame()
sleep_decoding = {0: 'Wake', 1: 'N1', 2:'N2', 3:'N3', 4:'REM', 9:'Error'}
for patient in patients:
    values, counts = np.unique(sleep[patient], return_counts=True)
    metrics = metrics.append({'id':patient, **{sleep_decoding[int(v)]:c for v,c in zip(values,counts)}}, ignore_index=True)
metrics = metrics.fillna(0)

In [ ]:
metrics['Total'] = metrics['Wake'] + metrics['N1'] + metrics['N2'] + metrics['N3'] + metrics['REM'] 

In [ ]:
metrics['Wake%'] = metrics['Wake']/metrics['Total']
metrics['Light%'] = (metrics['N1']+metrics['N2']) /metrics['Total']
metrics['Deep%'] = metrics['N3']/metrics['Total']
metrics['REM%'] = metrics['REM']/metrics['Total']

In [ ]:
metrics.sort_values(by='Deep%').tail(10)

In [ ]:
def duplicate(x):
    return np.array([[x_] * 2 for x_ in x]).flatten()


def hold_line(t, x):
    t = [i for i in t]
    x = [i for i in x]

    if len(t) - len(x) == 0:
        t.append(t[-1] + t[-1] - t[-2])
    if len(t) - len(x) != 1:
        raise ValueError()

    t_ = duplicate(t)[1:-1]
    x_ = duplicate(x)
    return t_, x_


def pretty_hypno(patient):
    sleep = dl.load_sleep(patient)
    sleep[sleep>4] = 0
#     sleep[sleep==4] = 3
    t = np.arange(0, len(sleep)*30, 30)/3600
#     plt.close('all')
    fig, ax = plt.subplots(1,1,figsize=(8,4))
    t, sleep = hold_line(t, sleep)
    ax.plot(t, sleep)
    ax.set_xlim([1, 3.5])
    ax.set_xticklabels(ax.get_xticks(), size=12)
#     ax.set_xticklabels(ax.get_xticks().astype(int), size=12)
    ax.set_yticks([0,1,2,3,4])
    ax.set_yticklabels(['Wake', "N1", "N2", "N3", "REM"],size=12)
    ax.set_xlabel("Time Sleeping (Hours)", size=12)
    ax.set_ylabel("Sleep Stage (AASM)", size=12)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plt.tight_layout()
    plt.show()

In [ ]:
metadata = dl.metadata
metadata['in_patients'] = [i in patients for i in metadata.id.values]
metadata = metadata[metadata.in_patients == True]
display(metadata.sort_values(by='ahi', ascending=False)[['id', 'ahi']].dropna().head(20))

In [ ]:
pretty_hypno('3121')
plt.savefig('/home/kkotzen/images/thesis/fig-high-ahi.pdf', bbox_inches='tight',pad_inches = 0)

# Compare Models Boxplot

In [ ]:
import pandas as pd
fe = pd.read_excel("/home/kkotzen/tmp/results/578982118a07416da5ec73ecf4d2676d.xlsx")
dts = pd.read_excel("/home/kkotzen/tmp/results/f296a562b5fa472fbee35da5d49648a8.xlsx")
wav = pd.read_excel("/home/kkotzen/tmp/results/66c27d8a81ee4a1e87ee3a4f8b0a14b8.xlsx")

df = pd.DataFrame({"BM-FE": fe["Kappa:"].values, "BM-DTS": dts["Kappa:"].values, "SleepPPG": wav["Kappa:"].values})
fig, ax = plt.subplots(1,1, figsize=(4,3))
df.boxplot(ax = ax, showfliers=False)
plt.grid(False)
ax.set_xlabel('Model')
ax.set_ylabel("κ")
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tight_layout()
plt.savefig('/home/kkotzen/images/thesis/fig-performance_boxplot_MESA.pdf', bbox_inches='tight',pad_inches = 0)

In [ ]:
df = pd.DataFrame({"BM-FE": fe["Kappa:"].values, "BM-DTS": dts["Kappa:"].values, "SleepPPG": wav["Kappa:"].values})
fig, ax = plt.subplots(1,1, figsize=(4,3))
df.plot.kde(ax = ax)
plt.grid(False)
ax.set_xlabel("κ")
ax.set_ylabel('Density')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlim([0,1])
plt.tight_layout()
plt.savefig('/home/kkotzen/images/thesis/fig-performance_kde_MESA.pdf', bbox_inches='tight',pad_inches = 0)

# Plot patients vs performance

In [ ]:
# %matplotlib widget
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from matplotlib import rcParams

labelsize = 10
rcParams['xtick.labelsize'] = labelsize
rcParams['ytick.labelsize'] = labelsize 
rcParams['axes.labelsize'] = labelsize 
plt.rcParams["figure.figsize"] = (3.5,3)

In [ ]:
#Add the variance
#USe the linked boxplot here!

x = [10,30,50,90,120,150,240]
y = [0.687, 0.695,0.71,0.719,0.729, 0.735,0.736]

fig, ax = plt.subplots(1,1, figsize=(3.5,2))

plt.plot(x, y, 'o')
plt.plot(x,y)      
ax.set_xlabel('Number of patients from CFS-train.')
ax.set_ylabel("κ")
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_ylim([0.65,0.75])
plt.tight_layout()
plt.savefig('/home/kkotzen/images/thesis/fig-transfer-learning-performance-by-patientst.pdf', bbox_inches='tight',pad_inches = 0)

In [ ]:
df = pd.DataFrame(np.zeros((100,100)))

In [ ]:
df

In [ ]:
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import cohen_kappa_score, confusion_matrix


def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]


def make_confusion_matrix(cf=[], y_reference=[], y_predicted=[],
                          group_names=None,
                          categories='auto',
                          count=True,
                          percent=True,
                          cbar=False,
                          xyticks=True,
                          xyplotlabels=True,
                          sum_stats=True,
                          precision_recall=True,
                          figsize=None,
                          cmap='Blues',
                          title=None):
    '''
    Modified from: https://github.com/DTrimarchi10/confusion_matrix/blob/master/cf_matrix.py
    This function will make a pretty plot of an sklearn Confusion Matrix cm using a Seaborn heatmap visualization.
    Arguments
    ---------
    cf:            confusion matrix to be passed in
    y_reference:   Ground truth Labels
    y_predicted:   Predicted Labels
    group_names:   List of strings that represent the labels row by row to be shown in each square.
    categories:    List of strings containing the categories to be displayed on the x,y axis. Default is 'auto'
    count:         If True, show the raw number in the confusion matrix. Default is True.
    normalize:     If True, show the proportions for each category. Default is True.
    cbar:          If True, show the color bar. The cbar values are based off the values in the confusion matrix.
                   Default is True.
    xyticks:       If True, show x and y ticks. Default is True.
    xyplotlabels:  If True, show 'True Label' and 'Predicted Label' on the figure. Default is True.
    sum_stats:     If True, display summary statistics below the figure. Default is True.
    figsize:       Tuple representing the figure size. Default will be the matplotlib rcParams value.
    cmap:          Colormap of the values displayed from matplotlib.pyplot.cm. Default is 'Blues'
                   See http://matplotlib.org/examples/color/colormaps_reference.html

    title:         Title for the heatmap. Default is None.
    '''

    def kappa():
        if len(y_predicted) > 0:
            return cohen_kappa_score(y_reference, y_predicted)
        else:
            return None

    def format_number(number):
        if number < 1000:
            return "{0:0.0f}".format(number)
        elif number < 1000000:
            return "{0:0.00f}K".format(number / 1000)
        else:
            return "{0:0.00f}M".format(number / 1000000)

    # Generate the confusion matrix if it does not already exist
    if len(cf) == 0:
        if len(y_reference) == 0 or len(y_predicted) == 0:
            raise ValueError(
                "You have to either specify a confusion matrix cf= or provide y_reference=y_reference and y_predicted=y_predicted.")
        try:
            cf = confusion_matrix(y_reference, y_predicted)
            cf_norm = confusion_matrix(y_reference, y_predicted, normalize='true')
        except:
            raise ValueError("Could not generate a confusion matrix from the y_reference and y_predicted provided.")
    else:
        cf_norm = cf/cf.sum(axis=1)

    # CODE TO GENERATE TEXT INSIDE EACH SQUARE
    blanks = ['' for i in range(cf.size)]

    if group_names and len(group_names) == cf.size:
        group_labels = ["{}\n".format(value) for value in group_names]
    else:
        group_labels = blanks

    if count:
        group_counts = [f"{format_number(value)}\n" for value in cf.flatten()]
    else:
        group_counts = blanks

    if percent:
        group_percentages = [f"{int(value*100)}%" for value in cf_norm.flatten()]
    else:
        group_percentages = blanks

    box_labels = [f"{v1}{v2}{v3}".strip() for v1, v2, v3 in zip(group_labels, group_counts, group_percentages)]
    box_labels = np.asarray(box_labels).reshape(cf.shape[0], cf.shape[1])

    # CODE TO GENERATE SUMMARY STATISTICS & TEXT FOR SUMMARY STATS
    precision = np.diagonal(cf) / np.sum(cf, axis=0)
    recall = np.diagonal(cf) / np.sum(cf, axis=1)
    accuracy = np.trace(cf) / float(np.sum(cf))
    count = np.sum(cf)

    # SET FIGURE PARAMETERS ACCORDING TO OTHER ARGUMENTS
    if figsize == None:
        # Get default figure size if not set
        figsize = plt.rcParams.get('figure.figsize')

    if xyticks == False:
        # Do not show categories if xyticks is False
        categories = False

    # MAKE THE HEATMAP VISUALIZATION
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    sns.heatmap(cf, annot=box_labels, fmt="", cmap=cmap, cbar=cbar, xticklabels=categories, yticklabels=categories,
                ax=ax, annot_kws={"fontsize":12})
    if xyplotlabels:
        ax.set_ylabel('Reference')
        ax.set_xlabel('Prediction')

    if title:
        fig.suptitle(title)

    if precision_recall:
        precision_ax = ax.secondary_xaxis('top')
        precision_ax.set_xticks([0.5, 1.5, 2.5, 3.5], minor=False)
        precision_labels = [f"{int(v*100)}%" for v in np.around(precision, decimals=2)]
        precision_ax.set_xticklabels(precision_labels)
        precision_ax.set_xlabel('PPV')

        recall_ax = ax.secondary_yaxis('right')
        recall_ax.set_yticks([0.5, 1.5, 2.5, 3.5], minor=False)
        recall_labels = [f"{int(v*100)}%" for v in np.around(recall, decimals=2)]
        recall_ax.set_yticklabels(recall_labels)
        recall_ax.set_ylabel('Sensativity')
        recall_ax.spines['right'].set_visible(False)
        recall_ax.spines['top'].set_visible(False)
        pass

    if sum_stats:
        stats_text = "ACC={:0.3f}, k={:0.3f}, Count={}".format(
            accuracy, kappa(), format_number(count))

        ax.set_title(stats_text)

    return  fig



In [ ]:
probs_file_fold = {}
probs_file_fold[1] = "/home/kkotzen/tmp/results/122d439c57bc49c19ed109e0ab57718f.pkl"
probs_file_fold[2] = "/home/kkotzen/tmp/results/122d439c57bc49c19ed109e0ab57718f.pkl"
probs_file_fold[3] = "/home/kkotzen/tmp/results/122d439c57bc49c19ed109e0ab57718f.pkl"
probs_file_fold[4] = "/home/kkotzen/tmp/results/122d439c57bc49c19ed109e0ab57718f.pkl"


sleep_predictions = np.array([])
sleep_labels = np.array([])
for i in range(4):
    predictions = pickle.load(open(probs_file_fold[i+1], "rb"))
    for i,patient in enumerate(predictions['patients']):
        sleep_predictions = np.append(sleep_predictions, [p for p in predictions["predictions"][i]])
        sleep_labels = np.append(sleep_labels, [p for p in predictions["labels"][i]])

    

In [ ]:
make_confusion_matrix(y_reference=sleep_labels, y_predicted=sleep_predictions,categories=['Wake', 'Light', 'Deep', 'REM'], figsize=(5, 4), cbar=False, title=False, sum_stats=True)
plt.savefig('/home/kkotzen/images/thesis/fig-confusion-matrix-TL-CFS.pdf', bbox_inches='tight',pad_inches = 0)

# Plot some waveforms

In [ ]:
import numpy as np
dl = MESAParser()

ppg1 = dl.load_signal('5073', 'PPG')


In [ ]:
max_level = np.quantile(np.abs(ppg), 0.99)
signal_99 = ppg[np.abs(ppg) < max_level]
rms = np.sqrt(np.sum(np.square(signal_99))/len(signal_99))
ppg_rms = ppg/(rms*10)
plt.close('all')
# plt.plot(ppg)
plt.plot(ppg_rms)
print(rms)

In [ ]:
plt.close('all')
ax = plt.subplot(2,1,1)
plt.plot(ppg)
ax.set_xticks([])
ax.set_yticks([])

ax2 = plt.subplot(2,1,2, sharex=ax)
plt.plot(ecg)
ax2.set_xticks([])
ax2.set_yticks([])
